In [1]:
#import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Input, Flatten, Dense, ReLU, BatchNormalization, Dropout
from tensorflow.keras import Model
from tensorflow.keras.losses import CategoricalCrossentropy
from ultralytics import YOLO
from keras.callbacks import ModelCheckpoint

from PIL import Image

from tensorflow.keras.applications import VGG19 as vgg

from matplotlib.pyplot import imshow


# This notebook is to train the final model for deployment into the app


In [3]:
#The data is read from the directory and each batch can be thought of as a set of labels and datapoints
#Might need to change the directory depending on where

#Seed 50 is decent performing
train = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=16,
    image_size=(500, 500),
   
    )



Found 164 files belonging to 3 classes.


In [4]:
train_processed = train.map(lambda x, y: (keras.applications.vgg19.preprocess_input(x), y))


In [5]:
#flatten the output
inputs = Input(shape=(500, 500, 3))

#set inference mode to be false, flatten the output and add dense layers before making the final prediction
vgg_output = keras.applications.VGG19(input_tensor=inputs,                                      
    include_top=False,
    weights="imagenet",

    input_shape=(500, 500, 3),
    pooling=None,


)
for layer in vgg_output.layers:
    layer.trainable = False
flatten = Flatten()(vgg_output.output)
dense1 = Dense(256, activation = 'relu', name = 'first_dense_layer')(flatten)
dropout1 = Dropout(0.25)(dense1)
batch_norm1 = BatchNormalization(name = 'batch_norm1')(dropout1)
dense2 = Dense(256, activation = 'relu', name = 'second_dense_layer')(batch_norm1)
dropout2 = Dropout(0.25)(dense2)
batch_norm1 = BatchNormalization(name = 'batch_norm2')(dropout2)
output = Dense(3, activation = 'softmax', name = 'output')(batch_norm1)
final_model = Model(inputs, output)


# In[ ]:



final_model.compile(optimizer = 'adam', loss = CategoricalCrossentropy(), metrics = ['accuracy'])
#save the model every epoch
checkpoint = ModelCheckpoint(filepath = 'vggemotion.keras')



In [6]:
final_model.fit(train_processed, epochs=40)


# In[ ]:


final_model.save('vggemotion_final.keras')

Epoch 1/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 145s 14s/step - accuracy: 0.3708 - loss: 1.6062
Epoch 2/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 127s 11s/step - accuracy: 0.6296 - loss: 0.9160
Epoch 3/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 132s 12s/step - accuracy: 0.6386 - loss: 0.7752
Epoch 4/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 114s 10s/step - accuracy: 0.7886 - loss: 0.5229
Epoch 5/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 93s 8s/step - accuracy: 0.8166 - loss: 0.4942
Epoch 6/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 132s 12s/step - accuracy: 0.8720 - loss: 0.3679
Epoch 7/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 147s 13s/step - accuracy: 0.8798 - loss: 0.2855
Epoch 8/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 216s 20s/step - accuracy: 0.9080 - loss: 0.2719
Epoch 9/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 225s 20s/step - accuracy: 0.8892 - loss: 0.2595
Epoch 10/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 209s 18s/step - accuracy: 0.9455 - loss: 0.1674
Epoch 11/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 212s 19s/step - accuracy: 0.9552 - loss: 0.1538
Epoch 12/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 210s 19s/step -